In [65]:
import pandas as pd

df = pd.read_csv("combined_credit_cards.csv")

df["combined_text"] = df.apply(lambda row: " ".join(row.values.astype(str)), axis=1)

print(df.head())



              Card Name Joining_Fee Renewal_Fee  \
0  SBI Card Miles Elite        4999        4999   
1        SBI Card PULSE       1,499       1,499   
2  SimplyCLICK SBI Card         499         499   
3     CASHBACK SBI Card         999         999   
4        SBI Card ELITE       4,999       4,999   

                                            Benefits  \
0  Get 5,000 Travel Credits as Welcome Gift, Earn...   
1  Annual Fee (one-time):Rs. 1,499 + Taxes, Annua...   
2  Get Amazon.in gift card worthRs. 500* on payme...   
3  5% cashback on online spends, 1% cashback on o...   
4  Welcome e-Gift Voucher worthRs. 5,000 on joini...   

                                            Features  \
0  {'Terms and Conditions': [], 'Welcome Gift': [...   
1  {'Terms and Conditions': ['For detailed Terms ...   
2  {'Contactless Advantage': ['Daily purchases no...   
3  {'Terms and Conditions': ['For detailed Terms ...   
4  {'Terms and Conditions': ['For detailed Terms ...   

                   

In [66]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

df["embeddings"] = df["combined_text"].apply(lambda x: embedding_model.embed_query(x))


In [67]:
from langchain.vectorstores import FAISS
from langchain.schema import Document

documents = [Document(page_content=row["combined_text"], metadata={"index": i}) for i, row in df.iterrows()]

vectorstore = FAISS.from_documents(documents, embedding_model)

vectorstore.save_local("combined_credit_cards")


In [68]:
import os
from google.colab import userdata

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get("HUGGINGFACEHUB_API_TOKEN")

from huggingface_hub import InferenceClient

client = InferenceClient(model="mistralai/Mistral-7B-Instruct-v0.3", token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))


In [70]:
query = input("Enter your query: ")

retrieved_docs = vectorstore.similarity_search(query, k=3)

context = "\n\n".join([doc.page_content for doc in retrieved_docs])

response = client.text_generation(f"Context:\n{context}\n\nQuestion: {query}")

print("\n🔹 AI Response:\n", response)

Enter your query: which card is better sbi or axis for shopping?

🔹 AI Response:
 
Answer: The comparison between SBI and Axis Bank credit cards for shopping depends on the specific card benefits and rewards offered. For instance, the South Indian Bank SBI Card PRIME offers 10 Reward Points per Rs. 100 spent on Dining, Groceries, Departmental stores and Movies, while the Axis Bank Buzz Credit Card offers 5 Reward Points per Rs. 100 spent on all purchases. Therefore, the South Indian Bank SBI Card PRIME might be a better choice for shopping at these categories. However, it's essential to consider other factors such as annual fees, welcome offers, and additional benefits before making a decision.
